In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataloaders import InfiniteDataLoader
from datasets import SubclassedDataset
from models import TransferModel18
from utils.image_data_utils import images_to_df, get_features
from utils.cluster_utils import train_erm_cluster, extract_features, features_to_df, split_features, check_cluster
from umap import UMAP
import numpy as np
import pandas as pd
import torch

In [3]:
def test_sc(tr_loader, cv_loader, tst_loader, images_df, split_path='./data/train_test_splits/LIDC_data_split.csv', split_num=None, get_cv_embeds=False, device='cpu'):

    model = TransferModel18(pretrained=True, freeze=False, device=device)
    train_erm_cluster(model, device=device, loaders=(
        tr_loader, cv_loader, tst_loader))

    noduleID, features = extract_features(
        model, images_df=images_df, device=device)

    df_features_all = features_to_df(noduleID, features)
  

    # features and corresponding malignancy, noduleID
    train_f, cv_test_f = split_features(
        df_features_all, split_path=split_path, split_num=split_num)

    reducer = UMAP(random_state=8)
    reducer.fit(train_f[0])

    train_e, cv_test_e = reducer.transform(
        train_f[0]), reducer.transform(cv_test_f[0])

    malig_max = np.argmax(check_cluster(train_e[train_f[1] > 1]))+2
    benig_max = np.argmax(check_cluster(train_e[train_f[1] <= 1]))+2

    return malig_max, benig_max, df_features_all


In [6]:
split_file = './data/train_test_splits/LIDC_data_split.csv' #'./data/train_test_splits/Nodule_Level_30Splits/nodule_split_all.csv'
split_df = pd.read_csv(split_file, index_col=0)

In [7]:
split_file = './data/train_test_splits/LIDC_data_split.csv' #'./data/train_test_splits/Nodule_Level_30Splits/nodule_split_all.csv'
split_df 

,noduleID,split,malignancy,malignancy_b,spic_groups
0,1,2,3,1,3
1,2,0,3,1,2
2,3,0,3,1,3
3,4,0,2,1,3
4,5,0,2,1,2
...,...,...,...,...,...
1483,2687,2,2,1,3
1484,2688,1,1,0,0
1485,2689,0,1,0,0
1486,2690,0,3,1,3


In [5]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
split_file = './data/train_test_splits/LIDC_data_split.csv' #'./data/train_test_splits/Nodule_Level_30Splits/nodule_split_all.csv'
split_df = pd.read_csv(split_file, index_col=0)
# split_num = 0

print('Loading images')
images_df = images_to_df()

df_clusters = pd.DataFrame()

train_data, cv_data, test_data = get_features(
    images=True, features=images_df, device=DEVICE, subclass='malignancy')

# datasets
tr = SubclassedDataset(*train_data)
cv = SubclassedDataset(*cv_data)
tst = SubclassedDataset(*test_data)

# dataloaders
tr_loader = InfiniteDataLoader(tr, batch_size=512)
cv_loader = InfiniteDataLoader(cv, len(cv))
tst_loader = InfiniteDataLoader(tst, len(tst))

for trial in range(30):
    print(f'==============  Trial {trial} ==============')

    torch.cuda.empty_cache()
    m, b, df_features = test_sc(tr_loader, cv_loader, tst_loader, images_df, device=DEVICE)
    torch.cuda.empty_cache()
    df_features.to_csv("./data/CNN_features/CNNeatures_{}.csv".format(trial))
    print(f'Malginant max sc: {m} Benign max sc:{b}')

Loading images


RuntimeError: CUDA out of memory. Tried to allocate 306.00 MiB (GPU 0; 8.00 GiB total capacity; 5.94 GiB already allocated; 0 bytes free; 6.21 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF